<a href="https://colab.research.google.com/github/rdsmaia/dim0494/blob/main/transformer_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import torch
from math import sqrt
from torch import nn
import torch.nn.functional as F

In [41]:
# hiperparâmetros (config)
class Params():
  embed_dim = 768                 # dimensão de cada vetor de entrada: x_t, t=1, ...,T (T é o número de vetores)
  hidden_size = embed_dim         # dimensão dos vetores de saída do transformador
  mlp_multi = 4                   # mlp_multi * model_dim é a dimensão das camadas feed-forward pontuais
  num_attention_heads = 12        # número de cabeças dos módulos de atenção múltipla, devem dividir model_dim
  hidden_dropout_prob = 0.2       # dropout das camadas FF
  max_position_embeddings = 2000  # numéro máximo de vetores na entrada
  num_hidden_layers = 12          # número de camadas do codificador
  intermediate_size = mlp_multi * hidden_size

config = Params()
assert config.hidden_size % config.num_attention_heads == 0

In [43]:
def scaled_dot_product_attention(query, key, value):
  '''
  Módulo de atenção simples, conforme mostrado aqui: https://arxiv.org/pdf/1706.03762

  Entrada:
    query - as consultas  (matriz Q)
    key   - as chaves (matriz K)
    value - os valores (matriz V)

  Saída:
    vetores contexto, dados por SOFTMAX(QK^T/sqrt(M_K))V
  '''
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
  weights = F.softmax(scores, dim=-1)
  return torch.bmm(weights, value)


class AttentionHead(nn.Module):
  '''
  Implementa uma cabeça de atenção, ou seja, determina as
  matrizes K, Q, V e chama o módulo de atenção simples.
  '''
  def __init__(self, embed_dim, head_dim):
    '''
    embed_dim: dimensão dos vetores de entrada.
    head_dim: dimensão dos vetores contexto (latentes) de cada cabeça
    '''
    super().__init__()
    self.q = nn.Linear(embed_dim, head_dim)
    self.k = nn.Linear(embed_dim, head_dim)
    self.v = nn.Linear(embed_dim, head_dim)

  def forward(self, hidden_state):
    '''
    Entrada:
      vetores latentes ou entrada.
    Saída:
      Novos vetores latentes.
    '''
    attn_outputs = scaled_dot_product_attention(
        self.q(hidden_state), self.k(hidden_state), self.v(hidden_state)
    )
    return attn_outputs


class MultiHeadAttention(nn.Module):
  '''
  Módulo de atenção múltiplo paralelo.
  '''
  def __init__(self, config):
    super().__init__()
    embed_dim = config.hidden_size
    num_heads = config.num_attention_heads
    head_dim = embed_dim // num_heads
    self.heads = nn.ModuleList(
        [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
    )
    self.output_linear = nn.Linear(embed_dim, embed_dim)

  def forward(self, hidden_state):
    '''
    Entrada:
      vetores latentes ou entrada.
    Saída:
      Novos vetores latentes.
    '''
    x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
    x = self.output_linear(x)
    return x

In [44]:
# vamos definir o MHA
mha = MultiHeadAttention(config)

In [45]:
# vamos criar uma entrada aleatória e passá-la pelo MHA
num_vectors = 10
B = 2  # batch size
X = torch.rand((B, num_vectors, config.embed_dim))
print(X.shape)

torch.Size([2, 10, 768])


In [46]:
# note que o número de vetores na saída é o mesmo que na entrada.
attn_output = mha(X)
print(attn_output.size())

torch.Size([2, 10, 768])


In [47]:
class FeedForward(nn.Module):
  '''
  Camada feed-forward pontual.
  '''
  def __init__(self, config):
    super().__init__()
    self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
    self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
    self.gelu = nn.GELU()
    self.dropout = nn.Dropout(config.hidden_dropout_prob)

  def forward(self, x):
    x = self.linear_1(x)
    x = self.gelu(x)
    x = self.linear_2(x)
    x = self.gelu(x)
    x = self.dropout(x)
    return x


In [48]:
# definimos a camada FF e passamos a saída do módulo MHA
feed_forward = FeedForward(config)
ff_output = feed_forward(attn_output)
print(ff_output.size())

torch.Size([2, 10, 768])


In [49]:
class TransformerEncoderLayer(nn.Module):
  '''
  Uma camanda de codificação, que envolve: MHA, LayerNorm e FF.
  '''
  def __init__(self, config):
    super().__init__()
    self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
    self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
    self.attention = MultiHeadAttention(config)
    self.feed_forward = FeedForward(config)

  def forward(self, x):
    hidden_state = self.layer_norm_1(x)
    x = x + self.attention(hidden_state)
    x = x + self.feed_forward(self.layer_norm_2(x))
    return x


In [50]:
# Vamos definir a nossa camada de codificação e passar nossa entrada por ela
encoder_layer = TransformerEncoderLayer(config)
Y = encoder_layer(X)
print(Y.shape)

torch.Size([2, 10, 768])


In [51]:
class Embeddings(nn.Module):
  '''
  Embeddings de posição e adição aos vetores de entrada.
  '''
  def __init__(self, config):
    super().__init__()
    self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
    self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
    self.dropout =nn.Dropout()

  def forward(self, input):
    # tamanho da sequência de entrada
    seq_length = input.size(1)
    # cria um vetor de posições e.g. (0, 1, 2, ..., T-1)
    position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
    # obtém os embeddings de posição
    position_embeddings = self.position_embeddings(position_ids)
    embeddings = input + position_embeddings
    embeddings = self.layer_norm(embeddings)
    embeddings = self.dropout(embeddings)
    return embeddings


In [52]:
# cria camada embedding e passa os vetores de entrada por ela
embedding_layer = Embeddings(config)
X_pos = embedding_layer(X)
print(X_pos.shape)

torch.Size([2, 10, 768])


In [53]:
class TransformerEncoder(nn.Module):
  '''
  Implementa um codificador (transformer)
  '''
  def __init__(self, config):
    super().__init__()
    self.embeddings = Embeddings(config)
    self.layers = nn.ModuleList(
        TransformerEncoderLayer(config) for _ in range(config.num_hidden_layers)
    )

  def forward(self, x):
    x = self.embeddings(x)
    for layer in self.layers:
      x = layer(x)
    return x


In [54]:
# define o codificador
encoder = TransformerEncoder(config)

In [55]:
# passa a entrada pelo codificador
Y = encoder(X)
print(Y.shape)

torch.Size([2, 10, 768])
